## requirements libs
### window
```
python-ffmpeg == 2.0.12
hidapi == 0.14.0.post4
pyserial == 3.5
```
pip install -e .

In [ ]:
# steam example
from device import DeviceGroupsWin
from device import VideoFFmpeg

HID_VID = "534D"
HID_PID = "2109"
Serial_port_VID = "1A86"
Serial_port_PID = "7523"
device_info_list = DeviceGroupsWin.search_phycial_device(Serial_port_VID, Serial_port_PID, HID_VID, HID_PID)
video_path = device_info_list[0]['camera_path']
audio_path = device_info_list[0]['audio_path']
video_url = "rtp://192.168.100.66:5000"
VideoFFmpeg.start_stream(video_path, audio_path, video_url, "windows")